In [1]:
import os
import time
from openai import OpenAI
import keyring

In [2]:
api_key = keyring.get_password(service_name='openai', username='key_for_mac')
client = OpenAI(
    api_key=api_key
)

In [3]:
assistant_name = "Power point generator"        # assistant name 
# output_file_name = "C:/Users/woojin/Documents/presentation01.pptx"    # for windows
output_file_name = "data/presentation01.pptx"       # for mac

# 구체적인 지시사항
assistant_instruction = r"파일을 만들어줘 {} 언제나. 너는 해당 주제의 전문가이자 PowerPoint 작성 전문가야. 배경, 색상, 글꼴 및 스타일은 현대적이고 읽기 쉽게 1-5개 슬라이드로 만들어줘. 콘텐츠를 매력적으로 만들어줘. 또한 파일을 다운로드할 수 있어야해".format(output_file_name)
prompt_user = "개인 기록을 목표로 등산을 연습하는 사람들을 위한 프레젠테이션을 만들어줘. 유용한 통찰력, 다양한 수준에 대한 교육 계획 및 교육 기간 전, 도중 및 후에 몇 가지 팁이 포함된 프레젠테이션을 만들어줘. 물론 통찰력도 제공해야해"

In [4]:
## assistant > thread > message > run

assistant = client.beta.assistants.create(
    name=assistant_name,
    instructions=assistant_instruction,
    tools=[{"type": "code_interpreter"}],
    model='gpt-4-1106-preview'
)

In [5]:
# 스레드는 어스시턴트와 사용자 간의 대화 세션
# 채팅 창에서 대화를 시작하면 스레드가 생성
# 해당 대화 중에 교환되는 모든 메시지가 동일한 스레드에서 진행
thread = client.beta.threads.create()

# 메시지는 스레드 내의 통신 단위
# 사용자 쿼리, 명령 또는 사용자의 어시스턴트 간에 교환되는 모든 콘텐츠일 수 있습니다.
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content=prompt_user
)

In [6]:
# 실행은 어시스턴트와의 단일 상호작용을 의미
# 실행은 사용자가 대화를 시작 (스레드 생성)할 때 시작되고 대화가 끝나면 종료됩니다.
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [7]:
timeout = 180
interval_time = 5
time_taken = 0
while time_taken < timeout:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    if run.status == 'completed':
        break  

in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


In [8]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_name = client.files.with_raw_response.retrieve_content(file_path)

In [9]:
with open(output_file_name, 'wb') as file:
    file.write(file_name.content)

In [10]:
client.files.delete(file_path)
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_JaTVWp9loUFWUA38tUzkOdni', deleted=True, object='assistant.deleted')